<a href="https://colab.research.google.com/github/Shufen-Yin/Artificial-Intelligence/blob/main/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from scipy import stats
from scipy.interpolate import interp1d
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Task 1 Data Collection
stocks_df = pd.read_csv("historical_stocks.csv")
prices_df = pd.read_csv("historical_stock_prices.csv")

In [3]:
# Check the stock metadata
print(stocks_df.head())
print(stocks_df.info())

# Check the stock prices
print(prices_df.head())
print(prices_df.info())


  ticker exchange                                    name             sector  \
0    PIH   NASDAQ  1347 PROPERTY INSURANCE HOLDINGS, INC.            FINANCE   
1  PIHPP   NASDAQ  1347 PROPERTY INSURANCE HOLDINGS, INC.            FINANCE   
2   TURN   NASDAQ                180 DEGREE CAPITAL CORP.            FINANCE   
3   FLWS   NASDAQ                 1-800 FLOWERS.COM, INC.  CONSUMER SERVICES   
4   FCCY   NASDAQ           1ST CONSTITUTION BANCORP (NJ)            FINANCE   

                     industry  
0  PROPERTY-CASUALTY INSURERS  
1  PROPERTY-CASUALTY INSURERS  
2  FINANCE/INVESTORS SERVICES  
3      OTHER SPECIALTY STORES  
4        SAVINGS INSTITUTIONS  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6460 entries, 0 to 6459
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ticker    6460 non-null   object
 1   exchange  6460 non-null   object
 2   name      6460 non-null   object
 3   sector    5020 non-null   

Ensure the dataset has no corrupted rows or misaligned columns.

Check for data types (numerical, categorical) and missing values.

In [4]:
# Clean whitespace
prices_df['date'] = prices_df['date'].astype(str).str.strip()
# Convert safely
prices_df['date'] = pd.to_datetime(prices_df['date'], errors='coerce')
# Drop the bad rows
prices_df = prices_df.dropna(subset=['date'])

# Merge
merged = prices_df.merge(stocks_df, on="ticker", how="left")
merged.index = prices_df.index
print("Merged dataset preview:")
merged.head()

Merged dataset preview:


,ticker,open,close,adj_close,low,high,volume,date,exchange,name,sector,industry
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900.0,2013-05-08,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800.0,2013-05-09,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100.0,2013-05-10,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400.0,2013-05-13,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100.0,2013-05-14,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE


In [5]:
# Advanc missing valure handling
print("Before handling missing values:")
print(merged.isnull().sum())

Before handling missing values:
ticker          0
open            0
close           0
adj_close       0
low             0
high            0
volume          0
date            0
exchange        0
name            0
sector       4490
industry     4490
dtype: int64


In [6]:
# Forward fill numeric columns
numeric_cols = merged.select_dtypes(include=[np.number]).columns
merged[numeric_cols] = merged[numeric_cols].interpolate(method='linear')

# Fill migssing categorical value with Unknown
cat_cols = merged.select_dtypes(include=['object']).columns
merged[cat_cols] = merged[cat_cols].fillna('Unknown')


1️⃣ Numeric Columns Imputation

Linear interpolation fills missing values in numeric columns based on the trend of surrounding values.

Purpose: Preserves temporal continuity in time-series data and avoids bias from mean/zero filling.

2️⃣ Categorical Columns Imputation

Missing values in categorical columns are replaced with 'Unknown'.

Purpose: Keeps all rows intact and prepares the dataset for encoding (e.g., one-hot encoding) without losing information.

3️⃣ Result Check

Use merged.isnull().sum() to verify that no missing values remain.

After this step, the dataset is ready for outlier handling, feature engineering, and scaling.

In [7]:
# Outlier detection & handling
# Using IOR for "close" and volume
def remove_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    # keep only rows within the acceptable range.

print("Before outlier removal: ",merged.shape)
merged = remove_outliers_iqr(merged, 'close')
merged = remove_outliers_iqr(merged, 'volume')
print("After outlier removal: ",merged.shape)


Before outlier removal:  (31703, 12)
After outlier removal:  (26732, 12)


1️⃣ Method Choice: IQR

The Interquartile Range (IQR) measures the spread of the middle 50% of the data.

Chosen over Z-score or mean-based methods because IQR is robust to extreme values and works well for non-normal or skewed distributions, like stock prices and trading volumes.

2️⃣ Outlier Detection Principle

Calculate Q1 (25th percentile) and Q3 (75th percentile), then IQR = Q3 − Q1.

Define bounds:

Lower bound = Q1 − 1.5 × IQR

Upper bound = Q3 + 1.5 × IQR

Values outside this range are considered potential outliers.

3️⃣ Outlier Handling

Keep only rows within the bounds; remove extreme values.

Reduces instability in model training, especially for linear models and distance-based models (KNN, regression, SVM).

4️⃣ Applicability

Ideal for financial, stock, or time-series data with natural noise and volatility.

Simple, efficient, and distribution-free, making it suitable for this project.

In [8]:
# Error Identification and Correction:
# Negative prices or volume
print("Before error correction:")
print(merged.describe())

Before error correction:
               open         close     adj_close           low          high  \
count  26732.000000  26732.000000  26732.000000  26732.000000  26732.000000   
mean      11.316096     11.310981      9.303130     11.144518     11.474519   
min        0.124128      0.123261      0.000015      0.123261      0.125867   
25%        4.520833      4.500000      2.853534      4.416670      4.625000   
50%       10.316250     10.300000      6.949924     10.100000     10.500000   
75%       15.690000     15.700000     13.311972     15.562500     15.812500   
max       36.430000     35.290001     35.290001     35.259998     36.810001   
std        8.429612      8.427710      8.510061      8.327145      8.522235   

              volume                           date  
count   26732.000000                          26732  
mean   139300.254377  2003-08-28 23:12:39.000448768  
min       100.000000            1980-03-17 00:00:00  
25%     25800.000000            1996-08-16 00:0

In [9]:
# Replace nevigatice value with Nan then interpolate
for col in ['close','high','low','close', 'volume']:
    merged.loc[merged[col] < 0, col] = np.nan
    merged[col] = merged[col].interpolate()
print("Any negative values left?")
print((merged[['close','high','low','close', 'volume']]<0).sum())

print("After error correction:")
print(merged.describe())

Any negative values left?
close     0
high      0
low       0
close     0
volume    0
dtype: int64
After error correction:
               open         close     adj_close           low          high  \
count  26732.000000  26732.000000  26732.000000  26732.000000  26732.000000   
mean      11.316096     11.310981      9.303130     11.144518     11.474519   
min        0.124128      0.123261      0.000015      0.123261      0.125867   
25%        4.520833      4.500000      2.853534      4.416670      4.625000   
50%       10.316250     10.300000      6.949924     10.100000     10.500000   
75%       15.690000     15.700000     13.311972     15.562500     15.812500   
max       36.430000     35.290001     35.290001     35.259998     36.810001   
std        8.429612      8.427710      8.510061      8.327145      8.522235   

              volume                           date  
count   26732.000000                          26732  
mean   139300.254377  2003-08-28 23:12:39.000448768  
min

1️⃣ Method Choice: Negative Value Detection & Interpolation

In stock datasets, prices and volume cannot be negative.

Negative values usually indicate data entry or collection errors.

If left uncorrected, they can distort statistics and predictive models.

2️⃣ Error Identification

Check columns close, high, low, volume for values < 0.

Mark negative values as NaN.

3️⃣ Error Correction

Replace negative values with NaN, then interpolate linearly.

Why interpolation?

Maintains temporal continuity in time-series data.

Avoids losing valuable rows, unlike deletion.

Better preserves trends than mean or zero filling.

4️⃣ Result Check

Use describe() to compare statistics before and after correction.

Use sum() to ensure all negative values are removed.

In [10]:
# Task 2 Data Transformation
# Feature Engineering:
merged['ma_7'] = merged['close'].rolling(7).mean()
merged['ma_30'] = merged['close'].rolling(30).mean()
merged['volatility_30'] = merged['close'].rolling(30).std()

# Daily return
merged['daily_return'] = merged['close'].pct_change()

# Future close price (for ML prediction)
merged['future_close'] = merged['close'].shift(+7)
merged.tail()


,ticker,open,close,adj_close,low,high,volume,date,exchange,name,sector,industry,ma_7,ma_30,volatility_30,daily_return,future_close
31696,GHDX,33.570000,33.270000,33.270000,32.990002,33.950001,100300.0,2017-07-19,NASDAQ,"GENOMIC HEALTH, INC.",HEALTH CARE,MEDICAL SPECIALITIES,15.594286,14.398787,11.473130,1.455351,13.5875
31697,RRC,11.458330,11.333330,10.116105,11.291670,11.666670,49300.0,1998-02-10,NYSE,RANGE RESOURCES CORPORATION,ENERGY,OIL & GAS PRODUCTION,16.570476,14.701231,11.259710,-0.659353,4.5000
31698,FLWS,2.380000,2.550000,2.550000,2.350000,2.550000,132900.0,2011-01-26,NASDAQ,"1-800 FLOWERS.COM, INC.",CONSUMER SERVICES,OTHER SPECIALTY STORES,15.429047,14.430676,11.455807,-0.775000,10.5400
31700,MHD,10.000000,9.970000,5.210537,9.860000,10.150000,22600.0,2008-12-01,NYSE,"BLACKROCK MUNIHOLDINGS FUND, INC.",Unknown,Unknown,12.200476,13.669009,10.938947,2.909804,32.5700
31701,RAVN,0.888889,0.916667,0.011638,0.888889,0.916667,65600.0,1991-04-05,NASDAQ,"RAVEN INDUSTRIES, INC.",CAPITAL GOODS,SPECIALTY CHEMICALS,10.870000,13.257731,11.184245,-0.908058,10.2300


1️⃣ Moving Average (MA)

ma_7: 7-day moving average, ma_30: 30-day moving average.

Why?

Smooths short-term price fluctuations.

Provides trend information to help the model understand price direction.

Combining short-term and long-term averages captures different levels of trends.

2️⃣ Volatility

volatility_30: 30-day rolling standard deviation of price.

Why?

Measures the magnitude of price fluctuations.

Captures market instability or risk, which is predictive of future price movements.

3️⃣ Daily Return

Calculated as (today_close - yesterday_close)/yesterday_close.

Why?

Converts absolute price into relative change, reducing scale differences between stocks or periods.

Helps capture short-term momentum.

4️⃣ Future Close Price (Label for ML)

future_close = close price 7 days ahead.

Why?

Serves as the target variable for supervised learning.

Enables machine learning models to predict future prices based on historical features.

Maintains the time-series sequence for proper forecasting.

In [11]:
# 2.2 DATA NORMALIZATION / STANDARDIZATION

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_cols = ['open','high','low','close','volume',
               'ma_7','ma_30','volatility_30','daily_return']

merged[scaled_cols] = scaler.fit_transform(merged[scaled_cols])

merged.head()

,ticker,open,close,adj_close,low,high,volume,date,exchange,name,sector,industry,ma_7,ma_30,volatility_30,daily_return,future_close
1,AHH,0.040798,0.028362,8.471151,0.042690,0.021765,0.901026,2013-05-09,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE,NaN,NaN,NaN,NaN,NaN
2,AHH,0.027748,0.034295,8.507822,0.042690,0.014724,0.909607,2013-05-10,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE,NaN,NaN,NaN,-0.321476,NaN
3,AHH,0.037239,0.040227,8.544494,0.048695,0.020591,0.053466,2013-05-13,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE,NaN,NaN,NaN,-0.321477,NaN
4,AHH,0.033680,0.025988,8.456484,0.042690,0.014724,0.295720,2013-05-14,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE,NaN,NaN,NaN,-0.322604,NaN
5,AHH,0.033680,0.034295,8.507822,0.047494,0.014724,-0.412560,2013-05-15,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE,NaN,NaN,NaN,-0.321342,NaN


1️⃣ Method Choice: StandardScaler

StandardScaler transforms numerical features to mean = 0 and standard deviation = 1.

Suitable for features with different scales, e.g., price (10–35), volume (tens of thousands), rolling averages, volatility.

2️⃣ Why Standardization?

Ensures all features are on the same scale, preventing large-scale features from dominating the model.

Crucial for distance-based models (KNN, SVM) and gradient-based optimization (neural networks).

Improves convergence speed and training stability.

3️⃣ Applicability

Applies to all numerical features, especially when their scales differ significantly.

Not strictly necessary for tree-based models (Random Forest, XGBoost), but does not harm.

In [13]:
# 2.3 ENCODING CATEGORICAL VARIABLES

# One-hot encoding for industry, sector, exchange
cat_features = ['sector','industry','exchange','name']
merged_encoded = pd.get_dummies(merged, columns=cat_features, drop_first=True)

merged_encoded.head()

,ticker,open,close,adj_close,low,high,volume,date,ma_7,ma_30,...,"name_BLACKROCK MUNIHOLDINGS FUND, INC.",name_BRF S.A.,"name_CARE.COM, INC.","name_GENOMIC HEALTH, INC.","name_GRAY TELEVISION, INC.",name_INVESTAR HOLDING CORPORATION,name_POWERSHARES DWA CONSUMER CYCLICALS MOMENTUM PORTFOLIO,name_RANGE RESOURCES CORPORATION,"name_RAVEN INDUSTRIES, INC.",name_VIAVI SOLUTIONS INC.
1,AHH,0.040798,0.028362,8.471151,0.042690,0.021765,0.901026,2013-05-09,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2,AHH,0.027748,0.034295,8.507822,0.042690,0.014724,0.909607,2013-05-10,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
3,AHH,0.037239,0.040227,8.544494,0.048695,0.020591,0.053466,2013-05-13,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
4,AHH,0.033680,0.025988,8.456484,0.042690,0.014724,0.295720,2013-05-14,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
5,AHH,0.033680,0.034295,8.507822,0.047494,0.014724,-0.412560,2013-05-15,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False


1️⃣ Method Choice: One-Hot Encoding

Converts categorical features (e.g., sector, industry, exchange, name) into multiple binary (0/1) columns.

drop_first=True avoids the dummy variable trap, preventing multicollinearity in linear models.

2️⃣ Why One-Hot Encoding?

Machine learning models cannot directly process text categories; numeric representation is required.

Retains all category information without introducing ordinal assumptions.

Suitable for most supervised learning models (regression, classification, neural networks).

3️⃣ Applicability

All discrete categorical features, especially those without natural order.

For tree-based models, drop_first is not critical, but for linear or distance-based models, it helps avoid redundancy.

In [17]:
# Task 3 Integration and Formatting for Modeling
# 3.1 CONSOLIDATE CLEAN DATASET
clean_df = merged_encoded.copy()
clean_df.dropna(inplace=True)

print("Final cleaned dataset shape:", clean_df.shape)
clean_df.head()

Final cleaned dataset shape: (26703, 51)


,ticker,open,close,adj_close,low,high,volume,date,ma_7,ma_30,...,"name_BLACKROCK MUNIHOLDINGS FUND, INC.",name_BRF S.A.,"name_CARE.COM, INC.","name_GENOMIC HEALTH, INC.","name_GRAY TELEVISION, INC.",name_INVESTAR HOLDING CORPORATION,name_POWERSHARES DWA CONSUMER CYCLICALS MOMENTUM PORTFOLIO,name_RANGE RESOURCES CORPORATION,"name_RAVEN INDUSTRIES, INC.",name_VIAVI SOLUTIONS INC.
30,AHH,-0.007841,-0.021475,8.163107,-0.001743,-0.020478,0.421138,2013-06-20,0.037043,0.133896,...,False,False,False,False,False,False,False,False,False,False
31,AHH,-0.017332,-0.004863,8.265792,-0.002944,-0.023999,0.980897,2013-06-21,0.026048,0.128910,...,False,False,False,False,False,False,False,False,False,False
32,AHH,-0.013773,-0.010796,8.229120,-0.053383,-0.021652,0.080530,2013-06-24,0.012554,0.122143,...,False,False,False,False,False,False,False,False,False,False
33,AHH,-0.004282,0.008190,8.346467,-0.019757,-0.008744,-0.288463,2013-06-25,0.009056,0.117335,...,False,False,False,False,False,False,False,False,False,False
34,AHH,0.014699,0.008190,8.346467,-0.029365,-0.001704,0.228391,2013-06-26,0.000559,0.114664,...,False,False,False,False,False,False,False,False,False,False


1️⃣ Method Choice: Data Consolidation

Combine all cleaned, feature-engineered, standardized, and encoded data into a final dataset.

Purpose: Ensures consistency and simplifies downstream modeling.

2️⃣ Drop Remaining Missing Values

Even after interpolation and filling, some features (e.g., rolling averages, volatility) may have NaN at the beginning.

Why drop?

Prevents errors or bias during model training.

Ensures every row has a complete feature vector.

In [19]:
print(clean_df.columns.tolist())


['ticker', 'open', 'close', 'adj_close', 'low', 'high', 'volume', 'date', 'ma_7', 'ma_30', 'volatility_30', 'daily_return', 'future_close', 'sector_CONSUMER NON-DURABLES', 'sector_CONSUMER SERVICES', 'sector_ENERGY', 'sector_FINANCE', 'sector_HEALTH CARE', 'sector_PUBLIC UTILITIES', 'sector_TECHNOLOGY', 'sector_Unknown', 'industry_COMPUTER MANUFACTURING', 'industry_COMPUTER SOFTWARE: PREPACKAGED SOFTWARE', 'industry_INVESTMENT MANAGERS', 'industry_MAJOR BANKS', 'industry_MEAT/POULTRY/FISH', 'industry_MEDICAL SPECIALITIES', 'industry_OIL & GAS PRODUCTION', 'industry_OTHER CONSUMER SERVICES', 'industry_OTHER SPECIALTY STORES', 'industry_REAL ESTATE', 'industry_SEMICONDUCTORS', 'industry_SPECIALTY CHEMICALS', 'industry_TELECOMMUNICATIONS EQUIPMENT', 'industry_Unknown', 'exchange_NYSE', 'name_8X8 INC', 'name_AMERICAN SOFTWARE, INC.', 'name_APOLLO GLOBAL MANAGEMENT, LLC', 'name_APPLE INC.', 'name_ARMADA HOFFLER PROPERTIES, INC.', 'name_BLACKROCK MUNIHOLDINGS FUND, INC.', 'name_BRF S.A.', 'n

In [23]:
# 3.2 TRAIN / VALIDATION / TEST SPLIT
# Predict future_close_7

from sklearn.model_selection import train_test_split

target_col = 'future_close'

X = clean_df.drop(columns=[target_col])
y = clean_df[target_col]


X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, shuffle=False
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, shuffle=False
)

print("Train shape:", X_train.shape)
print("Validation shape:", X_val.shape)
print("Test shape:", X_test.shape)

Train shape: (18692, 50)
Validation shape: (4005, 50)
Test shape: (4006, 50)


1️⃣ Train / Validation / Test Split

Split the dataset into:

Training set (70%): For model learning

Validation set (15%): For hyperparameter tuning and overfitting monitoring

Test set (15%): For final evaluation

2️⃣ Time-Series Consideration

Set shuffle=False to preserve chronological order.

Why?

Stock prices are time-dependent; shuffling would break the temporal relationship.

Ensures the model uses past data to predict future prices, reflecting realistic prediction scenarios.

3️⃣ Result Check

Use shape to verify each subset size.

After splitting, the dataset is ready for training, validation, and testing of predictive models.

In [25]:
# Save clean data:
clean_df.to_csv("clean_data.csv", index=False)
X_train.to_csv("x_train.csv")
y_train.to_csv("y_train.csv")
X_val.to_csv("x_val.csv")
y_val.to_csv("y_val.csv")
X_test.to_csv("x_test.csv")
y_test.to_csv("y_test.csv")
print("Files saved successfully!")


Files saved successfully!


1️⃣ Save the Cleaned Dataset

clean_data.csv contains the fully cleaned, feature-engineered, standardized, and encoded data.

Purpose: Serves as the foundation for analysis and ensures reproducibility of data preprocessing.

2️⃣ Save Train / Validation / Test Splits

Save X_train, y_train, X_val, y_val, X_test, y_test separately.

Why?

Maintains consistent splits for experiments and model comparisons.

Avoids re-splitting every time the Notebook runs, ensuring comparable results.

3️⃣ Benefits

Reproducibility: Others can reproduce the same results with identical data.

Flexibility: Models can directly load CSV files for training, hyperparameter tuning, or deployment.